In [1]:
import numpy as np
import pandas as pd
# Ejemplo de ajuste de parámetros en Python
import yfinance as yf

In [2]:
df = yf.Ticker('AAPL')
df = df.history(start='2015-01-01', end='2024-01-01')

### Definir función para calcular el rendimiento de la estrategia


In [3]:
def backtest_strategy(short_window, long_window):
    df['SMA_short'] = df['Close'].rolling(window=short_window).mean()
    df['SMA_long'] = df['Close'].rolling(window=long_window).mean()
    df['Signal'] = np.where(df['SMA_short'] > df['SMA_long'], 1, -1)
    df['Position'] = df['Signal'].shift()
    df['Strategy_Returns'] = df['Position'] * df['Close'].pct_change()
    return (df['Strategy_Returns'] + 1).cumprod().iloc[-1] -1 # Retorno total

### Definir el rango de parámetros a probar


In [4]:
short_windows = range(10, 100, 10) # De 10 a 90 días
long_windows = range(50, 200, 10) # De 50 a 190 días

### Probar todas las combinaciones de parámetros


In [5]:
best_performance = -np.inf
best_params = (None, None)

In [6]:
for short_window in short_windows:
    for long_window in long_windows:
        if short_window >= long_window:
            continue  # Evitar combinaciones donde la media corta sea mayor o igual a la larga
        performance = backtest_strategy(short_window, long_window)
        if performance > best_performance:
            best_performance = performance
            best_params = (short_window, long_window)

print(f"Mejores parámetros: SMA corta={best_params[0]}, SMA larga={best_params[1]}")
print(f"Mejor rendimiento: {best_performance:.2%}")

Mejores parámetros: SMA corta=80, SMA larga=180
Mejor rendimiento: 313.70%
